In [1]:
#pip install tensorflow

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## GB MODEL

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
# Hyperparameter Tuning using GridSearchCV
param_grid = {'n_estimators': [50, 100, 200],
              'learning_rate': [0.01, 0.1, 0.3],
              'max_depth': [3, 5, 8],
              'max_features': ['auto', 'log2', 'sqrt'],
              'loss': ['log_loss'],
              'subsample': [0.5, 1],
             
             }

In [16]:
gb = GradientBoostingClassifier()

In [17]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(gb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [ ]:
# Fit the GridSearch object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   4.1s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   4.6s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   4.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   4.2s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=0.5; total time=   3.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=1; total time=   3.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=auto, n_estimators=50, subsample=1; total time=   3.4s
[CV] END learning_rate=0.01, loss=log_loss, max_dept

[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=50, subsample=1; total time= 1.9min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=50, subsample=1; total time= 2.3min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=50, subsample=1; total time= 2.9min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=50, subsample=1; total time= 2.9min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=50, subsample=1; total time= 2.8min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=100, subsample=0.5; total time= 2.7min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=100, subsample=0.5; total time= 2.6min
[CV] END learning_rate=0.01, loss=log_loss, max_depth=3, max_features=sqrt, n_estimators=100, subsample=0.5; total time

[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=   8.7s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=  10.2s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=  10.5s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=  10.5s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=0.5; total time=   9.0s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=1; total time=  14.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=1; total time=  16.3s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=5, max_features=log2, n_estimators=100, subsample=1; t

[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=100, subsample=1; total time=   1.6s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=100, subsample=1; total time=   1.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=100, subsample=1; total time=   1.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=100, subsample=1; total time=   1.5s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=100, subsample=1; total time=   1.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=200, subsample=0.5; total time=   1.5s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=200, subsample=0.5; total time=   1.4s
[CV] END learning_rate=0.01, loss=log_loss, max_depth=8, max_features=auto, n_estimators=200, subsample=0.5; total

In [ ]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred = best_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# work progressing...........


In [ ]:
grid_search.cv_results_

In [ ]:
df = pd.DataFrame(grid_search.cv_results_)
df

In [ ]:
df[['param_var_smoothing', 'params', 'mean_test_score']]

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_